In [2]:
import os
import pandas as pd

DATAPATH = "../data"

In [3]:
smi = pd.read_csv(os.path.join(DATAPATH, "dilismiles.csv"))
padel = pd.read_csv(os.path.join(DATAPATH, "dili_padel_2d.csv"))

/tmp/ipykernel_7713/1034768844.py:2: DtypeWarning: Columns (423,424,425,426,427,428,429,430,431,477,478,479,480,481,482,483,484,485,540,541,542,543,544,545,546,547,548,603,604,605,606,607,608,609,610,611,1012,1013,1050,1052,1074,1083,1084,1121,1123,1145,1217,1218,1219,1220,1222,1223,1224,1226,1227,1229) have mixed types. Specify dtype option on import or set low_memory=False.
  padel = pd.read_csv(os.path.join(DATAPATH, "dili_padel_2d.csv"))


In [5]:
#compare that the order of molecules remains the same between both files

In [4]:
smi["outcome"] = padel["Outcome"]
smi["CommonName"]= padel["CommonName"]
smi.rename(columns={"col_smiles":"smiles"}, inplace=True)

In [5]:
from rdkit import Chem
st_dmi = []
inchikeys = []
for s in smi["smiles"]:
    mol = Chem.MolFromSmiles(s)
    inchikey = Chem.MolToInchiKey(mol)
    inchikeys += [inchikey]
smi["inchikey"] = inchikeys
smi.to_csv(os.path.join(DATAPATH,"dilismiles_with_outcome.csv"), index=False)

# Data Analysis